In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder

In [2]:
df = pd.read_csv('Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
labencoder = LabelEncoder()
df['Gender'] = labencoder.fit_transform(df['Gender'])

In [8]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [11]:
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [27]:
from sklearn.preprocessing import OneHotEncoder
onencoder = OneHotEncoder()

In [29]:
geoencoder = onencoder.fit_transform(df[['Geography']])
geoencoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [32]:
geography = pd.DataFrame(geoencoder.toarray(),columns=onencoder.get_feature_names_out(['Geography']))
geography.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [34]:
df = pd.concat([df.drop('Geography',axis=1),geography],axis=1)

In [35]:
import pickle
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(labencoder,file)

with open('one_hot_encoder.pkl','wb') as file:
    pickle.dump(onencoder,file)

In [37]:
X = df.drop('Exited',axis=1)
y = df['Exited']

In [38]:
scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [39]:
with open('standard_scalar.pkl','wb') as file:
    pickle.dump(scaler,file)

# **ANN Implementation**

In [60]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [55]:
model = Sequential([
    Dense(64,activation='relu',input_shape = (X_train.shape[1],)),    
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.01),loss='binary_crossentropy',metrics=['accuracy'])

In [74]:
log_dir = 'logs/fit' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [75]:
early_stoping = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [76]:
history = model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[tensorflow_callback,early_stoping])

Epoch 1/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8709 - loss: 0.3211 - val_accuracy: 0.8579 - val_loss: 0.3468
Epoch 2/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8651 - loss: 0.3223 - val_accuracy: 0.8618 - val_loss: 0.3438
Epoch 3/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8622 - loss: 0.3296 - val_accuracy: 0.8658 - val_loss: 0.3419
Epoch 4/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8677 - loss: 0.3191 - val_accuracy: 0.8597 - val_loss: 0.3468
Epoch 5/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.8663 - loss: 0.3206 - val_accuracy: 0.8618 - val_loss: 0.3452
Epoch 6/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8714 - loss: 0.3003 - val_accuracy: 0.8645 - val_loss: 0.3460
Epoch 7/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8699 - loss: 0.3105 - val_accuracy: 0.8652 - val_loss: 0.3452
Epoch 8/100
210/210 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8697 - loss: 0.3147 - val_accu

In [69]:
%load_ext tensorboard

In [79]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [80]:
model.save('model.h5')

In [81]:
from tensorflow.keras.models import load_model

In [103]:
my_model = load_model('model.h5')
with open('standard_scalar.pkl','rb') as file:
    standard_scalar = pickle.load(file)
    
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder = pickle.load(file)

with open('one_hot_encoder.pkl','rb') as file:
    one_hot_encoder = pickle.load(file)


In [87]:
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':600000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [98]:
geo_encoder = one_hot_encoder.transform([[input_data['Geography']]]).toarray()
data = pd.DataFrame([input_data])

In [99]:
input_data = pd.concat([data.drop('Geography',axis=1),geo_df],axis=1)

In [101]:
input_data['Gender'] = label_encoder.transform(input_data['Gender'])

In [102]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,600000,2,1,1,50000,1.0,0.0,0.0


In [106]:
input_data = standard_scalar.transform(input_data)
predication = my_model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [110]:
if  predication[0][0] > 0.5:
    print('The Customer is likely to churn')
else:
    print('The Customer Not to churn')

The Customer Not to churn
